<a href="https://colab.research.google.com/github/br4bit/Neural-Network-Training/blob/master/Training_Deep_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Addestramento rete neurale con centinaia di neuroni e migliaia di connessioni

Questo tipo di addestramento richiede un'analisi accurata dei seguenti problemi:



*  Primo: Problema del *Vanishing Gradients* e il correlato *Exploding Gradients* che si presenta nelle reti neurali molto profonde (più di 10 layers), questo problema rende difficile l'allenare i layers che si trovano nello strato inferiore.
*   Secondo: L'addestramento della rete neurale rallenta di molto con reti molto grandi.

*  Terzo: Un modello con milioni di parametri rischirebbe gravemente l'overfitting sul set di training.



# Problemi del Vanishing/Exploding Gradients

L'algoritmo di backpropagation lavora andando dall'output layer all'input layer, propagando l'errore del gradiente durante il tragitto. Una volta che l'algoritmo ha calcolato il gradiente della cost function utilizzando ogni parametro della rete, i parametri della rete vengono aggiornati (Gradient Descent step).

Sfortunatamente i gradienti calcolati diventano via via, sempre più piccoli man mano che l'algoritmo procede nei layers inferiori. Come risultato l'aggiornamento del Gradiente Descent, lascia i pesi delle connessioni nel layer inferiore virtalmente immutati (non li cambia), e l'addestramento non converge quindi ad una buona soluzione, da qui il nome *Vanishing Gradients*. In molti casi però, può accadere il contrario, ovvero i gradienti (derivate parziali) possono diventare molto grandi, e quindi causare anche l'incremento dei pesi in alcuni layer, di conseguenza l'algoritmo diverge (*Exploding Gradients*).
Più in generale le reti neurali, soffrono di gradienti instabili, dove diversi layers possono imparare a velocità ampliamente differenti. Questo è uno dei motivi per cui sono state abbandonate per molto tempo.

Ma nel 2010 grazie a : [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Che attraverso dei test con le funzioni di attivazione e inizializzazione dei pesi utilizzando una distribuzione normale con media 0 e deviazione standard a 1. In breve è stato dimostrato che con quel tipo di funzione di attivazione e schema di inizializzazione, la varianza dell'output di ogni layer era molto più grande della varianza dei proprio inputs. Andando in avanti nella rete la varianza continuava a crescere ad ogni layer fino a che la funzione di attivazione assumeva valori di saturazione nei layer in cima. Questo in realtà è aggravato dal fatto che la funzione logisitica ha una media di 0.5, non 0! (ad esempio la funzione tangente hyperbolica ha media 0 e si comporta molto bene rispetto alla logistic nelle reti molto grandi).

Diamo un'occhiata alla funzione logistica di attivazione (Sigmoid):

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/gradients-in-sigmoid-activation-functions.jpg)

E' possibile vedere che quando l'input assume valori molto grandi (sia positivi che negativi), la funzione satura o meglio assume valore 1 o 0, ed in quel la pendenza della retta tangente (la derivata) assume valori estremamente vicino allo 0.
Ne consegue che durante la fase di backpropagation, non esiste virtualmente nessun gradiente da propagare attraverso la rete, e quel piccolo gradiente che esiste si diluisce, mentre il backpropagation progredisce verso il basso attraverso i layers superiori, quindi non rimane nulla per i livelli inferiori.

## Xavier and He Inizializzazione

Dall'articolo Glorot e Bengio hanno proposto un modo per alleviare significamente questo problema. C'è bisogno che il segnale fluisca in modo appropriato in entrambe le direzioni: In avanti quando si fa una predizione e indietro quando si backpropagano i gradienti. Il segnale quindi non deve morire o assumere valori di saturazione.

Per far si che il segnale fluisca in modo corretto, l'autore ci dice che la varianza degli output di ogni layer deve essere uguale alla varianza degli input del layer : Var_Out(layer[1] = Var_In(layer[1])), questo per ogni layer. Inoltre ce bisogno che i gradienti abbiano prima e dopo l'attraversamento della rete nella direzione opposta, la stessa varianza. Non è possibile però garantire entrambi le condizioni a meno che i layers non abbiano lo stesso numero di connessioni di input e output, ma è possibile arrivare ad un compromesso che funziona molto bene in pratica:

Le connessioni dei pesi (matrice W) devono essere inizializzati in modo casuale secondo l'equazione di Xavier o Glorot.

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/normal-distribution-with-0-mean-and%20standard-deviation-formula.jpg)

Usando una delle due inizializzazione è possibile velocizzare l'addestramento. Inoltre questa strategia di inizializzazione nel caso si decida di utilizzare la funzione ReLU come funzione di attivazione è chiamata *He Initialization*.

Di default la funzione *tf.layers.dense()* usa la Xavier con distribuzione uniforme, è possibile cambiare anche con l'inizializzazione di He usando *variance_scaling_initializer()*

In [0]:
he_init = tf.contrib.layers.variance_scaling_initializer()

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu(), 
                          kernel_initializer=he_init, name="hidden1")


## Non Saturazione della funzione di attivazione

Sfortunatamente la funzione di attivazione ReLU non è perfetta, ma soffre di un problema conosciuto come *dying ReLus*, durante l'addestramento (training), alcuni neuroni è come se morissero, o meglio smettono di dare come output valori che sono diversi da 0, specialmente se si usa un learning rate molto alto. 

Per risolvere questo problema si usano delle varianti della funzione ReLU, come la *leaky ReLU*, questa funzione è definita come:

![alt text](http://img.thothchildren.com/09d44cab-d699-402a-af31-fdced34d94fd.png)
![alt text](http://wangxinliu.com/images/machine_learning/leakyrelu.png)

Dove α è un Hyperparametro e definisce quanto deve perdere la funzione o meglio qual è la pendenza della funzione per x<0 e tipicamente è impostato a 0.01. Questa piccola pendenza ci assicura che la Leaky ReLU non muoia mai.

Esistono molto varianti di questo genere, in particolare una nuova funzione di attivazione chiamata ELU (Exponential linear Unit) che ha superato la performance di tutte le varianti di ReLU nei vari esperimenti: tempo di training ridotto e performance migliorata sul test test. 

Ecco la sua definizione e rappresentazione:

![alt text](https://i.gyazo.com/7851e87ec39ce6a960e26709f39c1825.png)

![alt text](https://blog.paperspace.com/content/images/2018/06/ELU.png)

E' molto simile alla ReLU ma con alcune differenze:

* Primo: Ha una regione di valori negativi quando x<0, che permette alle unità di assumere valori di output vicino allo 0, il che aiuta con il problema del vanishing gradient. Hyperparamentro  α definisce il valore che la ELU deve approcciare quando x è un numero negativo molto grande, usualmente è impostato a 1.
* Secondo: Il suo gradiente per x<0 non è zero, quindi si evita il problema dei neuroni morti.
* Terzo: Quando Hyperparametro è uguale a 1 la funzione è molto regolare/morbida ovunque, incluso nell'intorno di x=0, questo aiuta molto a velocizzare il calcolo dei gradienti, evitando saltelli a destra e a sinistra nell'intorno di x=0.

L'unica pecca è che la ELU impiega molto tempo computazionale, dovuto alla funzione esponenziale, ma questo tempo è compensato dal fatto che durante il training converge molto più velocemente, quindi una rete con ReLU risulta di poco più veloce.

Quindi, quale funzione di attivazione utilizzare per gli hidden layers?

In generale ELU > leaky RELU (e varianti) > ReLU > tanh > logistic.

Se ci interessa il tempo di performance durante il runtime, la leaky ReLU è preferibile rispetto alla ELU.

Ma se il tempo di performance non ci interessa molto e abbiamo potenza computazione elevata è possibile utilizzare altre funzioni nel caso in cui: RReLU (overfitting) oppure PReLU se abbiamo un training set molto grande.

TensorFlow ci offre la *elu()* che è possibile utilizzare per costruire il nostro modello di rete neurale. Semplicemente basta impostare l'argomento del campo *activation*.

In [0]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")

Sfortunatamente TensorFlow non ha una funzione predefinita per la leaky ReLU, ma è possibile definirne una, nel seguente modo:

In [0]:
def leaky_relu(z, name=None):
  return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

## Batch Normalization